In [3]:
import json
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import re
import plotly.express as px
import plotly.graph_objects as go
import requests
from bs4 import BeautifulSoup as BS
from io import StringIO
from pandas.io.json import json_normalize
import glob
%matplotlib inline
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

### Recap of question for context

The dataset from data.nashville.gov includes geospatial information, which allows you to see where violations occurred geographically, but it does not provide information in regard to the specific businesses that were reported. In this part, you should explore the businesses and types of businesses that have been reported. To attempt to answer this, you have been provided data from the Google Places API. Each file is formatted as a json. The values are as follows:

**mapped_location**: The mapped location from the hubNashville dataset

**address**: The address from the hubNashville dataset

**results**: The first five results from a Google Maps API nearbysearch, ranked by proximity to the Mapped Location. See https://developers.google.com/places/web-service/search#PlaceSearchResponses for more details on the fields in the results.

Take these results and use them to match as many violations as possible to a business. Warning: it will not be possible to match all violations to a business, and there will be ambiguities when trying to do this match, so do not spend all of your time on this task. What do you find when you look into the types of businesses that have been reported for COVID violations?

In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

### Load all 49 JSON files with Google API data

#### Use glob to snag all the target files and merge their contents into one master JSON file

We could use glob to read in the files and store the contents of each as an entry in a list. But then we'd be adding yet another level of lists upon lists upon lists of dicts.

Instead of using .append(), use .extend().  This allows us to read in each JSON file, extract the contents of it and add it to the one before, avoiding the creation of even more layers! 

Then we take the resulting list and re-write it back into a JSON file.  If we try to use the list with pd.json_normalize() it won't work, so it must be written as JSON, saved and read back in. 

In [5]:
places=[] #create empty variable in which to store the contents of the 49 JSON files

for file in glob.glob('../data/google_places_results/results_*.json'): #for every file path in this glob list
    
    with open(file) as filein:  #open that JSON file
        
        places.extend(json.load(filein))  #load in the JSON file and tack on the contents to the helper list variable
        
with open('../data/google_places_results/google_results_merged.json','w') as outfile:   #Define the file path for the ouput and pass in the 'write' argument
    json.dump(places, outfile)     #Write the merged file as output 

**Read in the merged JSON file and check the output**

In [6]:
with open('../data/google_places_results/google_results_merged.json', "r") as fi:
    all_places = json.load(fi)

In [7]:
#check output
print(type(all_places),len(all_places))

<class 'list'> 4829


In [8]:
#Double check first entry in the resulting list
all_places[0]

all_places[0].keys()

dict_keys(['mapped_location', 'address', 'results'])

### Use pd.json_normalize() to turn the JSON file into a dataframe.

In [9]:
all_places = pd.json_normalize(all_places,   #pass in name of JSON file
                  record_path='results',     #Specify which path level of the JSON to unpack
                 meta=['mapped_location','address'])   #Specify any other elements outside that path to include

In [10]:
all_places.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24145 entries, 0 to 24144
Data columns (total 24 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   business_status                  24047 non-null  object 
 1   icon                             24145 non-null  object 
 2   name                             24145 non-null  object 
 3   photos                           13549 non-null  object 
 4   place_id                         24145 non-null  object 
 5   price_level                      3005 non-null   float64
 6   rating                           15251 non-null  float64
 7   reference                        24145 non-null  object 
 8   scope                            24145 non-null  object 
 9   types                            24145 non-null  object 
 10  user_ratings_total               15251 non-null  float64
 11  vicinity                         24145 non-null  object 
 12  geometry.location.

#### Clean & format the dataframe for analysis

In [11]:
all_places.columns

Index(['business_status', 'icon', 'name', 'photos', 'place_id', 'price_level',
       'rating', 'reference', 'scope', 'types', 'user_ratings_total',
       'vicinity', 'geometry.location.lat', 'geometry.location.lng',
       'geometry.viewport.northeast.lat', 'geometry.viewport.northeast.lng',
       'geometry.viewport.southwest.lat', 'geometry.viewport.southwest.lng',
       'opening_hours.open_now', 'plus_code.compound_code',
       'plus_code.global_code', 'permanently_closed', 'mapped_location',
       'address'],
      dtype='object')

In [12]:
# Drop unecessary columns
all_places = all_places.drop(columns=['place_id', 'reference','scope','plus_code.global_code', 
     'geometry.viewport.northeast.lat', 'geometry.viewport.northeast.lng',
       'geometry.viewport.southwest.lat', 'geometry.viewport.southwest.lng',
     'opening_hours.open_now','photos','permanently_closed'])

In [13]:
# Re-order columns for convenience
all_places=all_places[['mapped_location', 'address',
              'business_status',
               'name',
               'vicinity', 
                'geometry.location.lat',
                'geometry.location.lng', 
                'plus_code.compound_code',
                'types',
                'icon',
                'rating',
                'user_ratings_total',
                'price_level']]

In [14]:
#Re-name columns for clarity
all_places = all_places.rename(columns={"mapped_location":"violation_coord","address":"violation_address",
                           "vicinity":"google_address","geometry.location.lat":"google_lat",
                           "geometry.location.lng":"google_lng","plus_code.compound_code":"plus_code"})

#### The icon column links to the image file of the glyph assigned to that location by Google. We could possibly use this in analysis to make more accurate guesses about the type of establishment.

In [15]:
all_places.icon.unique()

array(['https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png',
       'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png',
       'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/shopping-71.png',
       'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/school-71.png',
       'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/worship_general-71.png',
       'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/hospital-71.png',
       'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/bar-71.png',
       'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
       'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/civic_building-71.png',
       'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/atm-71.png',
       'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/pharmacy-71.png',
       'https://maps.gstatic.com/mapfiles/pl

In [16]:
#Extract the icon name
all_places['icon'] = all_places['icon'].str.extract(r'^.*\/(.*)-71.png')

In [17]:
all_places['icon'].value_counts()

generic_business    13493
shopping             3746
restaurant           2428
school                802
atm                   484
worship_general       454
gas_station           403
civic_building        386
lodging               315
bar                   306
park                  219
cafe                  203
parking               182
pharmacy              165
bank                  150
bus                   134
cemetery_grave         67
hospital               58
museum                 32
police                 25
golf                   24
post_office            22
movies                 15
library                 8
geocode                 5
stadium                 4
worship_islam           3
camping                 3
worship_dharma          3
airport                 2
worship_jewish          2
worship_hindu           2
Name: icon, dtype: int64

### Find how many records match on violation address vs google address.

In [18]:
all_places.head()

,violation_coord,violation_address,business_status,name,google_address,google_lat,google_lng,plus_code,types,icon,rating,user_ratings_total,price_level
0,"(36.148664, -86.80740139999999)","2400 Elliston Pl, Nashville",OPERATIONAL,Martin's Bar-B-Que Joint,"2400 Elliston Place, Nashville",36.148618,-86.807367,"45XV+C3 Nashville, TN, USA","[restaurant, food, point_of_interest, establis...",restaurant,4.4,783.0,2.0
1,"(36.148664, -86.80740139999999)","2400 Elliston Pl, Nashville",OPERATIONAL,lunescent,"216 Reidhurst Avenue, Nashville",36.148448,-86.807697,"45XR+9W Nashville, TN, USA","[food, point_of_interest, establishment]",generic_business,5.0,3.0,NaN
2,"(36.148664, -86.80740139999999)","2400 Elliston Pl, Nashville",OPERATIONAL,Music row,"2400 West End Avenue, Nashville",36.148150,-86.807169,"45XV+74 Nashville, TN, USA","[electronics_store, home_goods_store, point_of...",shopping,NaN,NaN,NaN
3,"(36.148664, -86.80740139999999)","2400 Elliston Pl, Nashville",OPERATIONAL,Syntel Inc,"205 Reidhurst Avenue # 200, Nashville",36.148351,-86.808000,"45XR+8R Nashville, TN, USA","[point_of_interest, establishment]",generic_business,NaN,NaN,NaN
4,"(36.148664, -86.80740139999999)","2400 Elliston Pl, Nashville",OPERATIONAL,Nicholson Cleaners,"2410 Elliston Place, Nashville",36.148288,-86.807954,"45XR+8R Nashville, TN, USA","[laundry, point_of_interest, establishment]",generic_business,4.2,15.0,NaN


#### Test Fuzzy Matching

In [21]:
fuzzy_test = all_places.head(50)